In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import sys
sys.path.insert(0, 'drive/Graduation Project/Colaboratory')

In [5]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [6]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1019807728111074635]

In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [0]:
df = pd.read_csv('drive/Graduation Project/Colaboratory/user_data_anonymized.csv')
df = df.drop([df.columns[0]],axis=1)

In [9]:
df.head()

,user_id,username,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,type,ntrial,timestamp
0,0,User_1,0.032867,-0.011398,-0.989441,0.001340,-0.004726,0.000653,0,3,2018-03-15 21:49:02.853
1,0,User_1,0.036270,-0.646866,-0.749863,0.001340,-0.004726,0.000653,0,3,2018-03-15 21:49:02.876
2,0,User_1,0.036270,-0.646866,-0.749863,0.001340,-0.004726,0.000653,0,3,2018-03-15 21:49:02.903
3,0,User_1,-0.061005,-0.691040,-0.748917,0.320133,0.039149,-0.234852,0,3,2018-03-15 21:49:03.052
4,0,User_1,-0.140427,-0.746765,-0.741333,0.381414,0.046364,-0.343133,0,3,2018-03-15 21:49:03.074


In [0]:
cols = ['sensor_type','trial','name','type','set']
SEQ_LENGTH = 303
OVERLAP = 0
for i in range(SEQ_LENGTH):
    cols.append('x'+str(i))
moving_window_x = pd.DataFrame(columns = cols)

label = pd.DataFrame(columns = ['sensor_type','set','type1','type2','type3','type4'])
s_index = 0
for trial in np.sort(df['ntrial'].unique()):
    nt = df.loc[df['ntrial'] == trial]

    names = df['username'].unique()
    for name in names:
        n = nt.loc[nt['username'] == name]
        
        types = np.array([1,2,3,4])
        for type_ in types:
            t = n.loc[n['type'] == type_]
            
            sensors = t.columns[2:8]
            for sensor in sensors:  
                s = t
                
                if trial == 5: text="test"
                else: text="train"
                    
                row = np.array([sensor])
                row = np.append(row,[trial])
                row = np.append(row,[name])
                row = np.append(row,[type_])
                row = np.append(row,[text])
                if s[sensor].values.shape[0] > SEQ_LENGTH:
                    row = np.append(row,s[sensor].values[:SEQ_LENGTH].astype(float))
                    #print(s[sensor].values[:SEQ_LENGTH])
                elif s[sensor].values.shape[0] <= SEQ_LENGTH:
                    padded_row = np.append(s[sensor].values[:].astype(float),np.array([s[sensor].values[-1].astype(float) for i in range(SEQ_LENGTH-s[sensor].values.shape[0])]))
                    row = np.append(row,padded_row)
                moving_window_x = moving_window_x.append(pd.DataFrame([row],columns=moving_window_x.columns))
                
                row_label = np.array([sensor])
                row_label = np.append(row_label,[text])
                row_label = np.append(row_label,np.zeros((types.shape[0],)))
                row_label[type_+1] = 1.0
                label = label.append(pd.DataFrame([row_label],columns=label.columns))
                
    s_index += s[sensor].shape[0]

In [0]:
moving_window_x['index'] = [i for i in range(moving_window_x.shape[0])]
moving_window_x = moving_window_x.set_index(['index'])

label['index'] = [i for i in range(label.shape[0])]
label = label.set_index(['index'])

In [0]:
train_len = int(moving_window_x.loc[moving_window_x['set'] == 'train'].shape[0]/6)
test_len = int(moving_window_x.loc[moving_window_x['set'] == 'test'].shape[0]/6)

In [0]:
x_train = np.zeros((train_len,SEQ_LENGTH,6))
y_train = np.zeros((train_len,4))

x_test = np.zeros((test_len,SEQ_LENGTH,6))
y_test = np.zeros((test_len,4))

In [0]:
shuffle_data = np.zeros((train_len+test_len,SEQ_LENGTH,6+4))

In [15]:
for i,sensor in enumerate(moving_window_x['sensor_type'].unique()):
    s_train = moving_window_x.loc[moving_window_x['sensor_type'] == sensor]
    s_train = s_train.loc[s_train['set'] == "train"]
    s_test = moving_window_x.loc[moving_window_x['sensor_type'] == sensor]
    s_test = s_test.loc[s_test['set'] == "test"]

    x_train[:,:,i] = s_train.drop(['sensor_type','trial','name','type','set'],axis=1)
    x_test[:,:,i] = s_test.drop(['sensor_type','trial','name','type','set'],axis=1)
    if i == 0:
        l = label.loc[label['sensor_type'] == sensor]
        l_train = l.loc[l['set'] == "train"]
        l_test = l.loc[l['set'] == "test"]
        
        y_train[:,:] = l_train.drop(['sensor_type','set'],axis=1)
        y_test[:,:] = l_test.drop(['sensor_type','set'],axis=1)

print("x_train.shape: "+str(x_train.shape))
print("y_train.shape: "+str(y_train.shape))
print("x_test.shape: "+str(x_test.shape))
print("y_test.shape: "+str(y_test.shape))

x_train.shape: (32, 303, 6)
y_train.shape: (32, 4)
x_test.shape: (8, 303, 6)
y_test.shape: (8, 4)


In [16]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import History
import seaborn as sn
import time

Using TensorFlow backend.


In [18]:
y_train_ = np.zeros((32,303,4))
y_train_[:,0,:] = y_train

y_test_ = np.zeros((8,303,4))
y_test_[:,0,:] = y_test

shuffle_data[:32,:,:6] = x_train
shuffle_data[:32,:,6:] = y_train_

shuffle_data[32:,:,:6] = x_test
shuffle_data[32:,:,6:] = y_test_


while True:
  np.random.shuffle(shuffle_data)

  x_train = shuffle_data[:32,:,:6]
  x_test = shuffle_data[32:,:,:6]
  y_train = shuffle_data[:32,0,6:].reshape(32,4)
  y_test = shuffle_data[32:,0,6:].reshape(8,4)
  if np.sum(y_test[:,0]) == 2:
      if np.sum(y_test[:,1]) == 2:
          if np.sum(y_test[:,2]) == 2:
              if np.sum(y_test[:,3]) == 2:
                  break


batch_size = 1
timesteps = SEQ_LENGTH
data_dim = 6

model = Sequential()
model.add(LSTM(128, return_sequences=True, 
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(64))
model.add(Dense(32))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


st = time.time()
history = model.fit(x_train, y_train,
          batch_size=batch_size, epochs=100, shuffle=False,
          validation_data=(x_test, y_test),verbose=1)
ft = time.time()

print("batch size : %d -> %d seconds for training with CPU"%(batch_size,ft-st))


Train on 32 samples, validate on 8 samples
Epoch 1/100
32/32 [==============================] - 8s 261ms/step - loss: 1.4872 - acc: 0.1562 - val_loss: 1.4585 - val_acc: 0.1250
Epoch 2/100
32/32 [==============================] - 7s 222ms/step - loss: 1.3743 - acc: 0.2812 - val_loss: 1.4401 - val_acc: 0.0000e+00
Epoch 3/100
32/32 [==============================] - 7s 224ms/step - loss: 1.4050 - acc: 0.3438 - val_loss: 1.4673 - val_acc: 0.0000e+00
Epoch 4/100
32/32 [==============================] - 7s 222ms/step - loss: 1.3124 - acc: 0.3750 - val_loss: 1.4486 - val_acc: 0.0000e+00
Epoch 5/100
32/32 [==============================] - 7s 223ms/step - loss: 1.3181 - acc: 0.4688 - val_loss: 1.4844 - val_acc: 0.1250
Epoch 6/100
32/32 [==============================] - 7s 224ms/step - loss: 1.0655 - acc: 0.3750 - val_loss: 0.9584 - val_acc: 0.3750
Epoch 7/100
 6/32 [====>.........................] - ETA: 5s - loss: 1.8252 - acc: 0.3333

32/32 [==============================] - 7s 222ms/step - loss: 1.4889 - acc: 0.2812 - val_loss: 1.3412 - val_acc: 0.1250
Epoch 8/100
32/32 [==============================] - 7s 223ms/step - loss: 1.0210 - acc: 0.5000 - val_loss: 0.9280 - val_acc: 0.3750
Epoch 9/100
32/32 [==============================] - 7s 225ms/step - loss: 1.2870 - acc: 0.4688 - val_loss: 1.5933 - val_acc: 0.3750
Epoch 10/100
32/32 [==============================] - 7s 225ms/step - loss: 1.1685 - acc: 0.4062 - val_loss: 1.0533 - val_acc: 0.3750
Epoch 11/100
32/32 [==============================] - 7s 222ms/step - loss: 0.8645 - acc: 0.6875 - val_loss: 1.1243 - val_acc: 0.5000
Epoch 12/100
32/32 [==============================] - 7s 221ms/step - loss: 0.9824 - acc: 0.4062 - val_loss: 1.8043 - val_acc: 0.2500
Epoch 13/100
21/32 [==================>...........] - ETA: 2s - loss: 1.0551 - acc: 0.6667

32/32 [==============================] - 7s 225ms/step - loss: 1.2876 - acc: 0.4688 - val_loss: 1.5222 - val_acc: 0.1250
Epoch 14/100
32/32 [==============================] - 7s 223ms/step - loss: 0.9990 - acc: 0.5938 - val_loss: 1.2074 - val_acc: 0.3750
Epoch 15/100
32/32 [==============================] - 7s 224ms/step - loss: 0.9093 - acc: 0.6250 - val_loss: 0.9587 - val_acc: 0.3750
Epoch 16/100
32/32 [==============================] - 7s 224ms/step - loss: 0.8906 - acc: 0.5312 - val_loss: 0.8046 - val_acc: 0.3750
Epoch 17/100
32/32 [==============================] - 7s 222ms/step - loss: 0.7296 - acc: 0.5312 - val_loss: 0.7649 - val_acc: 0.5000
Epoch 18/100
32/32 [==============================] - 7s 223ms/step - loss: 0.6940 - acc: 0.5938 - val_loss: 1.2184 - val_acc: 0.5000
Epoch 19/100
21/32 [==================>...........] - ETA: 2s - loss: 0.7929 - acc: 0.4762

32/32 [==============================] - 7s 223ms/step - loss: 0.9216 - acc: 0.4062 - val_loss: 1.1941 - val_acc: 0.3750
Epoch 20/100
32/32 [==============================] - 7s 224ms/step - loss: 0.9402 - acc: 0.5625 - val_loss: 1.3834 - val_acc: 0.5000
Epoch 21/100
32/32 [==============================] - 7s 229ms/step - loss: 0.6943 - acc: 0.5312 - val_loss: 0.7997 - val_acc: 0.3750
Epoch 22/100
32/32 [==============================] - 7s 225ms/step - loss: 0.7012 - acc: 0.6250 - val_loss: 0.9147 - val_acc: 0.5000
Epoch 23/100
32/32 [==============================] - 7s 225ms/step - loss: 0.6786 - acc: 0.6562 - val_loss: 0.8891 - val_acc: 0.3750
Epoch 24/100
32/32 [==============================] - 7s 224ms/step - loss: 0.8284 - acc: 0.5625 - val_loss: 0.8627 - val_acc: 0.3750
Epoch 25/100
21/32 [==================>...........] - ETA: 2s - loss: 0.7297 - acc: 0.6667

32/32 [==============================] - 7s 227ms/step - loss: 0.7657 - acc: 0.6562 - val_loss: 0.8248 - val_acc: 0.3750
Epoch 26/100
32/32 [==============================] - 7s 224ms/step - loss: 1.4650 - acc: 0.4688 - val_loss: 1.2446 - val_acc: 0.3750
Epoch 27/100
32/32 [==============================] - 7s 225ms/step - loss: 1.2931 - acc: 0.2812 - val_loss: 1.1241 - val_acc: 0.1250
Epoch 28/100
32/32 [==============================] - 7s 227ms/step - loss: 1.1743 - acc: 0.4688 - val_loss: 1.0947 - val_acc: 0.2500
Epoch 29/100
32/32 [==============================] - 7s 224ms/step - loss: 1.0885 - acc: 0.4375 - val_loss: 1.0473 - val_acc: 0.2500
Epoch 30/100
32/32 [==============================] - 7s 227ms/step - loss: 1.0758 - acc: 0.4688 - val_loss: 1.1679 - val_acc: 0.1250
Epoch 31/100
21/32 [==================>...........] - ETA: 2s - loss: 0.9274 - acc: 0.6667

...

32/32 [==============================] - 7s 225ms/step - loss: 1.3496 - acc: 0.3438 - val_loss: 1.4865 - val_acc: 0.0000e+00
Epoch 74/100
32/32 [==============================] - 7s 225ms/step - loss: 1.3405 - acc: 0.3438 - val_loss: 1.4977 - val_acc: 0.0000e+00
Epoch 75/100
32/32 [==============================] - 7s 227ms/step - loss: 1.3227 - acc: 0.4062 - val_loss: 1.4734 - val_acc: 0.0000e+00
Epoch 76/100
32/32 [==============================] - 7s 224ms/step - loss: 1.3099 - acc: 0.3750 - val_loss: 1.4621 - val_acc: 0.0000e+00
Epoch 77/100
32/32 [==============================] - 7s 222ms/step - loss: 1.3115 - acc: 0.4062 - val_loss: 1.4537 - val_acc: 0.0000e+00
Epoch 78/100
32/32 [==============================] - 7s 224ms/step - loss: 1.3018 - acc: 0.3750 - val_loss: 1.5012 - val_acc: 0.0000e+00
Epoch 79/100
18/32 [===============>..............] - ETA: 2s - loss: 1.1919 - acc: 0.4444

32/32 [==============================] - 7s 222ms/step - loss: 1.2649 - acc: 0.4375 - val_loss: 1.4994 - val_acc: 0.0000e+00
Epoch 80/100
32/32 [==============================] - 7s 222ms/step - loss: 1.2453 - acc: 0.4375 - val_loss: 1.5275 - val_acc: 0.0000e+00
Epoch 81/100
32/32 [==============================] - 7s 225ms/step - loss: 1.2191 - acc: 0.4375 - val_loss: 1.3827 - val_acc: 0.2500
Epoch 82/100
32/32 [==============================] - 7s 227ms/step - loss: 1.2833 - acc: 0.3438 - val_loss: 1.4416 - val_acc: 0.2500
Epoch 83/100
32/32 [==============================] - 7s 225ms/step - loss: 1.2329 - acc: 0.3125 - val_loss: 1.5632 - val_acc: 0.0000e+00
Epoch 84/100
32/32 [==============================] - 7s 228ms/step - loss: 1.2032 - acc: 0.4375 - val_loss: 1.4086 - val_acc: 0.2500
Epoch 85/100
19/32 [================>.............] - ETA: 2s - loss: 1.0713 - acc: 0.4737

32/32 [==============================] - 7s 223ms/step - loss: 1.1577 - acc: 0.4688 - val_loss: 1.2797 - val_acc: 0.2500
Epoch 86/100
32/32 [==============================] - 7s 226ms/step - loss: 1.1880 - acc: 0.4375 - val_loss: 1.5481 - val_acc: 0.0000e+00
Epoch 87/100
32/32 [==============================] - 7s 225ms/step - loss: 1.3929 - acc: 0.1562 - val_loss: 1.3979 - val_acc: 0.2500
Epoch 88/100
32/32 [==============================] - 7s 227ms/step - loss: 1.3159 - acc: 0.2812 - val_loss: 1.4258 - val_acc: 0.2500
Epoch 89/100
32/32 [==============================] - 7s 229ms/step - loss: 1.2668 - acc: 0.2500 - val_loss: 1.4503 - val_acc: 0.0000e+00
Epoch 90/100
32/32 [==============================] - 7s 225ms/step - loss: 1.2333 - acc: 0.4375 - val_loss: 1.4862 - val_acc: 0.0000e+00
Epoch 91/100
20/32 [=================>............] - ETA: 2s - loss: 1.1105 - acc: 0.4500

32/32 [==============================] - 7s 229ms/step - loss: 1.2197 - acc: 0.4375 - val_loss: 1.4365 - val_acc: 0.2500
Epoch 92/100
32/32 [==============================] - 7s 227ms/step - loss: 1.2232 - acc: 0.4062 - val_loss: 1.5582 - val_acc: 0.0000e+00
Epoch 93/100
32/32 [==============================] - 7s 226ms/step - loss: 1.2019 - acc: 0.4375 - val_loss: 1.3582 - val_acc: 0.2500
Epoch 94/100
32/32 [==============================] - 7s 223ms/step - loss: 1.1592 - acc: 0.5000 - val_loss: 1.4322 - val_acc: 0.2500
Epoch 95/100
32/32 [==============================] - 7s 225ms/step - loss: 1.2458 - acc: 0.3750 - val_loss: 1.5552 - val_acc: 0.0000e+00
Epoch 96/100
32/32 [==============================] - 7s 229ms/step - loss: 1.1711 - acc: 0.5000 - val_loss: 1.2961 - val_acc: 0.2500
Epoch 97/100
20/32 [=================>............] - ETA: 2s - loss: 1.0822 - acc: 0.4000

32/32 [==============================] - 7s 223ms/step - loss: 1.1936 - acc: 0.3750 - val_loss: 1.4353 - val_acc: 0.1250
Epoch 98/100
32/32 [==============================] - 7s 223ms/step - loss: 1.4235 - acc: 0.3125 - val_loss: 1.4189 - val_acc: 0.2500
Epoch 99/100
32/32 [==============================] - 7s 225ms/step - loss: 1.3629 - acc: 0.1875 - val_loss: 1.4335 - val_acc: 0.0000e+00
Epoch 100/100
32/32 [==============================] - 7s 226ms/step - loss: 1.3161 - acc: 0.2500 - val_loss: 1.4836 - val_acc: 0.0000e+00
batch size : 1 -> 724 seconds for training with CPU


In [19]:
y_train_ = np.zeros((32,303,4))
y_train_[:,0,:] = y_train

y_test_ = np.zeros((8,303,4))
y_test_[:,0,:] = y_test

shuffle_data[:32,:,:6] = x_train
shuffle_data[:32,:,6:] = y_train_

shuffle_data[32:,:,:6] = x_test
shuffle_data[32:,:,6:] = y_test_


while True:
  np.random.shuffle(shuffle_data)

  x_train = shuffle_data[:32,:,:6]
  x_test = shuffle_data[32:,:,:6]
  y_train = shuffle_data[:32,0,6:].reshape(32,4)
  y_test = shuffle_data[32:,0,6:].reshape(8,4)
  if np.sum(y_test[:,0]) == 2:
      if np.sum(y_test[:,1]) == 2:
          if np.sum(y_test[:,2]) == 2:
              if np.sum(y_test[:,3]) == 2:
                  break


batch_size = 8
timesteps = SEQ_LENGTH
data_dim = 6

model = Sequential()
model.add(LSTM(128, return_sequences=True, 
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(64))
model.add(Dense(32))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


st = time.time()
history = model.fit(x_train, y_train,
          batch_size=batch_size, epochs=100, shuffle=False,
          validation_data=(x_test, y_test),verbose=1)
ft = time.time()

print("batch size : %d -> %d seconds for training with CPU"%(batch_size,ft-st))


Train on 32 samples, validate on 8 samples
Epoch 1/100
32/32 [==============================] - 2s 64ms/step - loss: 1.4193 - acc: 0.0938 - val_loss: 1.3552 - val_acc: 0.2500
Epoch 2/100
32/32 [==============================] - 1s 36ms/step - loss: 1.3729 - acc: 0.2812 - val_loss: 1.3438 - val_acc: 0.3750
Epoch 3/100
32/32 [==============================] - 1s 36ms/step - loss: 1.3499 - acc: 0.3125 - val_loss: 1.3270 - val_acc: 0.3750
Epoch 4/100
32/32 [==============================] - 1s 35ms/step - loss: 1.3219 - acc: 0.3125 - val_loss: 1.2893 - val_acc: 0.3750
Epoch 5/100
32/32 [==============================] - 1s 36ms/step - loss: 1.2793 - acc: 0.3438 - val_loss: 1.2213 - val_acc: 0.3750
Epoch 6/100
32/32 [==============================] - 1s 36ms/step - loss: 1.2156 - acc: 0.5000 - val_loss: 1.1464 - val_acc: 0.2500
Epoch 7/100
32/32 [==============================] - 1s 36ms/step - loss: 1.1297 - acc: 0.5312 - val_loss: 1.0099 - val_acc: 0.5000
Epoch 8/100
32/32 [==============

32/32 [==============================] - 1s 35ms/step - loss: 0.5828 - acc: 0.6875 - val_loss: 0.5921 - val_acc: 0.7500
Epoch 18/100
32/32 [==============================] - 1s 36ms/step - loss: 0.5638 - acc: 0.6875 - val_loss: 0.6104 - val_acc: 0.5000
Epoch 19/100
32/32 [==============================] - 1s 36ms/step - loss: 0.5065 - acc: 0.7812 - val_loss: 0.5448 - val_acc: 0.5000
Epoch 20/100
32/32 [==============================] - 1s 36ms/step - loss: 0.4624 - acc: 0.8438 - val_loss: 0.5168 - val_acc: 0.6250
Epoch 21/100
32/32 [==============================] - 1s 35ms/step - loss: 0.4244 - acc: 0.8438 - val_loss: 0.5063 - val_acc: 0.6250
Epoch 22/100
32/32 [==============================] - 1s 36ms/step - loss: 0.3199 - acc: 0.9375 - val_loss: 0.4269 - val_acc: 0.7500
Epoch 23/100
32/32 [==============================] - 1s 36ms/step - loss: 0.2202 - acc: 0.9688 - val_loss: 0.3639 - val_acc: 0.8750
Epoch 24/100
32/32 [==============================] - 1s 36ms/step - loss: 0.1852 

32/32 [==============================] - 1s 36ms/step - loss: 0.3071 - acc: 0.9062 - val_loss: 0.5880 - val_acc: 0.7500
Epoch 34/100
32/32 [==============================] - 1s 36ms/step - loss: 0.2208 - acc: 0.9688 - val_loss: 0.4453 - val_acc: 0.7500
Epoch 35/100
32/32 [==============================] - 1s 36ms/step - loss: 0.1648 - acc: 0.9688 - val_loss: 0.4548 - val_acc: 0.7500
Epoch 36/100
32/32 [==============================] - 1s 35ms/step - loss: 0.1532 - acc: 0.9688 - val_loss: 0.3361 - val_acc: 0.8750
Epoch 37/100
32/32 [==============================] - 1s 36ms/step - loss: 0.4329 - acc: 0.9062 - val_loss: 0.6081 - val_acc: 0.6250
Epoch 38/100
32/32 [==============================] - 1s 35ms/step - loss: 1.5176 - acc: 0.6875 - val_loss: 1.8702 - val_acc: 0.5000
Epoch 39/100
32/32 [==============================] - 1s 35ms/step - loss: 1.2948 - acc: 0.6562 - val_loss: 1.3497 - val_acc: 0.6250
Epoch 40/100
32/32 [==============================] - 1s 36ms/step - loss: 1.0239 

32/32 [==============================] - 1s 36ms/step - loss: 0.4502 - acc: 0.8750 - val_loss: 0.8892 - val_acc: 0.6250
Epoch 51/100
32/32 [==============================] - 1s 36ms/step - loss: 0.2690 - acc: 0.9375 - val_loss: 0.4592 - val_acc: 0.7500
Epoch 52/100
32/32 [==============================] - 1s 36ms/step - loss: 0.3583 - acc: 0.8750 - val_loss: 0.7392 - val_acc: 0.3750
Epoch 53/100
32/32 [==============================] - 1s 36ms/step - loss: 0.6371 - acc: 0.8125 - val_loss: 0.8615 - val_acc: 0.5000
Epoch 54/100
32/32 [==============================] - 1s 36ms/step - loss: 0.4061 - acc: 0.8438 - val_loss: 1.1447 - val_acc: 0.6250
Epoch 55/100
32/32 [==============================] - 1s 36ms/step - loss: 0.1730 - acc: 0.9688 - val_loss: 0.4448 - val_acc: 0.7500
Epoch 56/100
32/32 [==============================] - 1s 36ms/step - loss: 0.2133 - acc: 0.9375 - val_loss: 0.4318 - val_acc: 0.7500
Epoch 57/100
32/32 [==============================] - 1s 36ms/step - loss: 0.1252 

32/32 [==============================] - 1s 36ms/step - loss: 0.5325 - acc: 0.7812 - val_loss: 0.6269 - val_acc: 0.6250
Epoch 68/100
32/32 [==============================] - 1s 36ms/step - loss: 0.3574 - acc: 0.9062 - val_loss: 0.9512 - val_acc: 0.6250
Epoch 69/100
32/32 [==============================] - 1s 36ms/step - loss: 0.3547 - acc: 0.9375 - val_loss: 0.8849 - val_acc: 0.7500
Epoch 70/100
32/32 [==============================] - 1s 36ms/step - loss: 0.3257 - acc: 0.8438 - val_loss: 0.7215 - val_acc: 0.7500
Epoch 71/100
32/32 [==============================] - 1s 36ms/step - loss: 0.3033 - acc: 0.8438 - val_loss: 0.8126 - val_acc: 0.6250
Epoch 72/100
32/32 [==============================] - 1s 36ms/step - loss: 0.3335 - acc: 0.9062 - val_loss: 0.9451 - val_acc: 0.5000
Epoch 73/100
32/32 [==============================] - 1s 36ms/step - loss: 0.3976 - acc: 0.8750 - val_loss: 0.8221 - val_acc: 0.5000
Epoch 74/100
32/32 [==============================] - 1s 36ms/step - loss: 0.4513 

32/32 [==============================] - 1s 36ms/step - loss: 0.2898 - acc: 0.8750 - val_loss: 0.9562 - val_acc: 0.6250
Epoch 85/100
32/32 [==============================] - 1s 36ms/step - loss: 0.2840 - acc: 0.9062 - val_loss: 0.8155 - val_acc: 0.6250
Epoch 86/100
32/32 [==============================] - 1s 36ms/step - loss: 0.1713 - acc: 1.0000 - val_loss: 0.4425 - val_acc: 0.8750
Epoch 87/100
32/32 [==============================] - 1s 35ms/step - loss: 0.1444 - acc: 0.9688 - val_loss: 0.5054 - val_acc: 0.7500
Epoch 88/100
32/32 [==============================] - 1s 36ms/step - loss: 0.1353 - acc: 0.9688 - val_loss: 0.6386 - val_acc: 0.6250
Epoch 89/100
32/32 [==============================] - 1s 36ms/step - loss: 0.1293 - acc: 0.9688 - val_loss: 0.7505 - val_acc: 0.5000
Epoch 90/100
32/32 [==============================] - 1s 35ms/step - loss: 0.1638 - acc: 0.9688 - val_loss: 0.4679 - val_acc: 0.8750
Epoch 91/100
32/32 [==============================] - 1s 35ms/step - loss: 0.2588 